In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from sqlalchemy import create_engine
import numpy as np

## Extract the greatest books ever written using BeautifulSoup from:
https://www.britannica.com/list/12-novels-considered-the-greatest-book-ever-written

In [2]:
# URL of page to be scraped
url = "https://www.britannica.com/list/12-novels-considered-the-greatest-book-ever-written"

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
# Explore the BeautifulSoup object and retrive the book titles into a DataFrame. 
titles = soup.find_all('h2')
greatest_ever_written = []

for title in titles:
    greatest_ever_written.append(title.text)

greatest_ever_written_df = pd.DataFrame(greatest_ever_written,columns=['title'])

#Add a column to identify those books that are best ever written according to Britannic Encyclopedia.
greatest_ever_written_df['best_ever_written']= True
greatest_ever_written_df

,title,best_ever_written
0,Anna Karenina,True
1,To Kill a Mockingbird,True
2,The Great Gatsby,True
3,One Hundred Years of Solitude,True
4,A Passage to India,True
5,Invisible Man,True
6,Don Quixote,True
7,Beloved,True
8,Mrs. Dalloway,True
9,Things Fall Apart,True


## Extract books rating information from CSV file found in:  
https://www.kaggle.com/jealousleopard/goodreadsbooks

In [4]:
# Open CSV file and convert it to pandas DataFrame
csv_file = "./Resources/books.csv"
books_csv_df = pd.read_csv(csv_file)
books_csv_df.count()

bookID                11123
title                 11123
authors               11123
average_rating        11123
isbn                  11123
isbn13                11123
language_code         11123
  num_pages           11123
ratings_count         11123
text_reviews_count    11123
publication_date      11123
publisher             11123
dtype: int64

### Transform: Cleaning Rating Data frame 

In [5]:
# new data frame with split value columns 
new = books_csv_df["title"].str.split('[-(]', n = 1, expand = True) 
author_new = books_csv_df["authors"].str.split('/', n = 1, expand = True) 
  
# making separate first name column from new data frame 
books_csv_df["title"]= new[0] 
books_csv_df["authors"]= author_new[0] 

# df display 
books_csv_df.to_csv('books_df.csv')
books_csv_df

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half,J.K. Rowling,4.57,439785960,9.780000e+12,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix,J.K. Rowling,4.49,439358078,9.780000e+12,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets,J.K. Rowling,4.42,439554896,9.780000e+12,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban,J.K. Rowling,4.56,043965548X,9.780000e+12,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1,J.K. Rowling,4.78,439682584,9.780000e+12,eng,2690,41428,164,9/13/2004,Scholastic
...,...,...,...,...,...,...,...,...,...,...,...,...
11118,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann,4.06,1560254416,9.780000e+12,eng,512,156,20,12/21/2004,Da Capo Press
11119,45633,You Bright and Risen Angels,William T. Vollmann,4.08,140110879,9.780000e+12,eng,635,783,56,12/1/1988,Penguin Books
11120,45634,The Ice,William T. Vollmann,3.96,140131965,9.780000e+12,eng,415,820,95,8/1/1993,Penguin Books
11121,45639,Poor People,William T. Vollmann,3.72,60878827,9.780000e+12,eng,434,769,139,2/27/2007,Ecco


In [6]:
# Select only the information needed from the csv_df
books_review_df = books_csv_df[['title', 'authors','average_rating', 'ratings_count', 'text_reviews_count']].copy()
books_review_df = books_review_df.rename(columns={'  num_pages': 'num_pages', 'Author(s)':'authors'})
books_review_df

,title,authors,average_rating,ratings_count,text_reviews_count
0,Harry Potter and the Half,J.K. Rowling,4.57,2095690,27591
1,Harry Potter and the Order of the Phoenix,J.K. Rowling,4.49,2153167,29221
2,Harry Potter and the Chamber of Secrets,J.K. Rowling,4.42,6333,244
3,Harry Potter and the Prisoner of Azkaban,J.K. Rowling,4.56,2339585,36325
4,Harry Potter Boxed Set Books 1,J.K. Rowling,4.78,41428,164
...,...,...,...,...,...
11118,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann,4.06,156,20
11119,You Bright and Risen Angels,William T. Vollmann,4.08,783,56
11120,The Ice,William T. Vollmann,3.96,820,95
11121,Poor People,William T. Vollmann,3.72,769,139


In [7]:
# Select the publisher with more ratings_count per book
ratigns_df = books_review_df.sort_values(['title', 'ratings_count'], ascending=[True, False])
ratigns_df_clean = ratigns_df.drop_duplicates(subset='title', keep="first")
ratigns_df_clean.to_csv('ever_written.csv')
ratigns_df_clean

,title,authors,average_rating,ratings_count,text_reviews_count
1847,said the shotgun to the head.,Saul Williams,4.22,2762,214
4072,$30 Film School: How to Write Direct Produce...,Michael W. Dean,3.49,30,4
1572,'Salem's Lot,Stephen King,4.25,84123,571
3137,1 000 Places to See Before You Die,Patricia Schultz,3.85,36303,439
2343,10 lb Penalty,Dick Francis,3.90,3490,177
...,...,...,...,...,...
8316,鋼之鍊金術師 6,Hiromu Arakawa,4.58,5,0
8322,鋼之鍊金術師 7,Hiromu Arakawa,4.57,5,0
8319,鋼之鍊金術師 9,Hiromu Arakawa,4.57,4,0
4268,魔戒二部曲：雙城奇謀,J.R.R. Tolkien,4.44,24,0


## Extract Best Seller Books from a website using pandas. 
https://en.wikipedia.org/wiki/List_of_best-selling_books

In [8]:
#Extract HTML tables into DataFrames
html_link = "https://en.wikipedia.org/wiki/List_of_best-selling_books"
best_selling=pd.read_html(html_link)

total_tables_found=len(best_selling)
print(f"There's a Total of {total_tables_found} tables found on the Wikipedia HTML")

print("The useful set of tables range from 1-5")

There's a Total of 17 tables found on the Wikipedia HTML
The useful set of tables range from 1-5


In [9]:
#Loop to retrieve and merge each Table(df)
best_selling_df=best_selling[1]

for i in range(2,6):
    best_selling_df=best_selling_df.append(best_selling[i],ignore_index=True)
    
best_selling_df

,Book,Author(s),Original language,First published,Approximate sales,Genre
0,A Tale of Two Cities,Charles Dickens,English,1859,200 million[15],Historical fiction
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,150 million[16][17][18][19][20][21],fantasy (picture book)
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120 million[9][22],"fantasy, mystery"
3,The Hobbit,J. R. R. Tolkien,English,1937,100+ million [15],fantasy
4,And Then There Were None,Agatha Christie,English,1939,100 million[15],mystery
...,...,...,...,...,...,...
160,The Front Runner,Patricia Nell Warren,English,1974,10 million[169],NaN
161,The Goal,Eliyahu M. Goldratt,English,1984,10 million[170],NaN
162,Fahrenheit 451,Ray Bradbury,English,1953,10 million[171],NaN
163,Angela's Ashes,Frank McCourt,English,1996,10 million[172],NaN


### Transform: Cleaning Best Selling Books Data Frame 

In [10]:
#Clean Approximate sales
best_selling_df['Approximate sales']= best_selling_df['Approximate sales'].str.split(' million').str[0]
best_selling_df['Approximate sales']= best_selling_df['Approximate sales'].str.split('–').str[0]
best_selling_df['Approximate sales']= best_selling_df['Approximate sales'].str.split('+').str[0]

best_selling_df.at[best_selling_df.index[best_selling_df['Approximate sales']=='>80'],['Approximate sales']]= 80

best_selling_df

,Book,Author(s),Original language,First published,Approximate sales,Genre
0,A Tale of Two Cities,Charles Dickens,English,1859,200,Historical fiction
1,The Little Prince (Le Petit Prince),Antoine de Saint-Exupéry,French,1943,150,fantasy (picture book)
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120,"fantasy, mystery"
3,The Hobbit,J. R. R. Tolkien,English,1937,100,fantasy
4,And Then There Were None,Agatha Christie,English,1939,100,mystery
...,...,...,...,...,...,...
160,The Front Runner,Patricia Nell Warren,English,1974,10,NaN
161,The Goal,Eliyahu M. Goldratt,English,1984,10,NaN
162,Fahrenheit 451,Ray Bradbury,English,1953,10,NaN
163,Angela's Ashes,Frank McCourt,English,1996,10,NaN


In [11]:
# Cleaning First published
best_selling_df['First published']=best_selling_df['First published'].astype('str')
best_selling_df.at[best_selling_df.index[best_selling_df['First published'] =='1925-1929'],['First published']]= '1925'


best_selling_df['First published'].unique()

array(['1859', '1943', '1997', '1937', '1939', '1791', '1950', '1887',
       '1881', '2003', '1998', '1999', '2000', '2005', '2007', '1988',
       '1951', '1992', '1880', '1984', '1967', '1955', '1946', '1908',
       '1877', '1980', '1975', '1972', '1976', '1952', '1902', '1970',
       '1969', '1899', '1960', '1979', '1991', '1932', '1869', '1947',
       '1977', '2002', '1966', '1925', '1936', '1938', '1949', '2009',
       '2008', '1945', '1788', '1934', '1989', '1935', '1993', '2012',
       '2010', '1957', '2015', '1929', '1974', '1958', '2004', '1971',
       '1986', '1885', '1982', '1813', '1923', '1963', '2006', '1973',
       '1965', '1964', '1968', '1981', '1995', '2011', '1994', '1942',
       '1985', '2001', '1933', '1962', '1987', '1956', '1948', '1304',
       '2018', '1961', '1978', '1953', '1996'], dtype=object)

In [12]:
#Fix Book names where there are two titles(English and Original language)
best_selling_df['Book']= best_selling_df['Book'].str.split('(').str[0]
best_selling_df.tail()

,Book,Author(s),Original language,First published,Approximate sales,Genre
160,The Front Runner,Patricia Nell Warren,English,1974,10,NaN
161,The Goal,Eliyahu M. Goldratt,English,1984,10,NaN
162,Fahrenheit 451,Ray Bradbury,English,1953,10,NaN
163,Angela's Ashes,Frank McCourt,English,1996,10,NaN
164,The Story of My Experiments with Truth,Mohandas Karamchand Gandhi,Gujarati,1925,10,NaN


In [13]:
#Fix Author "E.B. White; illustrated by Garth Williams"
best_selling_df.at[best_selling_df.index[best_selling_df['Author(s)'] =='E.B. White; illustrated by Garth Williams'],['Author(s)']]= 'E.B. White'

In [14]:
# Check for repeatedness on Languages
print (best_selling_df['Original language'].unique())


['English' 'French' 'Chinese' 'Italian' 'Portuguese' 'Spanish'
 'Swiss German' 'Norwegian' 'Russian' 'Dutch' 'Swedish' 'Japanese' 'Hindi'
 'German' 'Czech' 'Yiddish' 'Gujarati']


In [15]:
#Change Swiss German language to German
best_selling_df.at[best_selling_df.index[best_selling_df['Original language'] =='Swiss German'],['Original language']]= 'German'
best_selling_df['Original language'].unique()

array(['English', 'French', 'Chinese', 'Italian', 'Portuguese', 'Spanish',
       'German', 'Norwegian', 'Russian', 'Dutch', 'Swedish', 'Japanese',
       'Hindi', 'Czech', 'Yiddish', 'Gujarati'], dtype=object)

In [16]:
best_selling_df['Genre'].unique()

array(['Historical fiction', 'fantasy (picture book)', 'fantasy, mystery',
       'fantasy', 'mystery', 'family saga', 'adventure',
       'mystery thriller', 'coming-of-age', 'romance', 'historical',
       'self-improvement', 'magic realism', 'general',
       "children's fiction", 'manual', "children's literature",
       'historical novel, mystery', 'war, thriller', nan], dtype=object)

In [17]:
best_selling_df = best_selling_df.replace({'Genre': {"Historical fiction": "fiction", "fantasy (picture book)": "fantasy", 
                                                     "fantasy, mystery":"fantasy", "mystery thriller":"mystery", 
                                                     "children's fiction":"children's literature",
                                                     "historical novel, mystery":"mystery","war, thriller":"thriller" }})
best_selling_df

,Book,Author(s),Original language,First published,Approximate sales,Genre
0,A Tale of Two Cities,Charles Dickens,English,1859,200,fiction
1,The Little Prince,Antoine de Saint-Exupéry,French,1943,150,fantasy
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120,fantasy
3,The Hobbit,J. R. R. Tolkien,English,1937,100,fantasy
4,And Then There Were None,Agatha Christie,English,1939,100,mystery
...,...,...,...,...,...,...
160,The Front Runner,Patricia Nell Warren,English,1974,10,NaN
161,The Goal,Eliyahu M. Goldratt,English,1984,10,NaN
162,Fahrenheit 451,Ray Bradbury,English,1953,10,NaN
163,Angela's Ashes,Frank McCourt,English,1996,10,NaN


In [18]:
# Change column names to match SQL column titles
best_selling_df=best_selling_df.rename(columns={'Book':'title',
                               'Author(s)':'authors',
                               'Original language':'original_language',
                               'First published':'first_published',
                               'Approximate sales':'approximate_sales',
                               'Genre':'genre'})
best_selling_df

,title,authors,original_language,first_published,approximate_sales,genre
0,A Tale of Two Cities,Charles Dickens,English,1859,200,fiction
1,The Little Prince,Antoine de Saint-Exupéry,French,1943,150,fantasy
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120,fantasy
3,The Hobbit,J. R. R. Tolkien,English,1937,100,fantasy
4,And Then There Were None,Agatha Christie,English,1939,100,mystery
...,...,...,...,...,...,...
160,The Front Runner,Patricia Nell Warren,English,1974,10,NaN
161,The Goal,Eliyahu M. Goldratt,English,1984,10,NaN
162,Fahrenheit 451,Ray Bradbury,English,1953,10,NaN
163,Angela's Ashes,Frank McCourt,English,1996,10,NaN


In [19]:
#Add a column to identify those books that are best ever written according to Britannic Encyclopedia.
best_selling_df['best_seller']= True
best_selling_df


,title,authors,original_language,first_published,approximate_sales,genre,best_seller
0,A Tale of Two Cities,Charles Dickens,English,1859,200,fiction,True
1,The Little Prince,Antoine de Saint-Exupéry,French,1943,150,fantasy,True
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120,fantasy,True
3,The Hobbit,J. R. R. Tolkien,English,1937,100,fantasy,True
4,And Then There Were None,Agatha Christie,English,1939,100,mystery,True
...,...,...,...,...,...,...,...
160,The Front Runner,Patricia Nell Warren,English,1974,10,NaN,True
161,The Goal,Eliyahu M. Goldratt,English,1984,10,NaN,True
162,Fahrenheit 451,Ray Bradbury,English,1953,10,NaN,True
163,Angela's Ashes,Frank McCourt,English,1996,10,NaN,True


### Transform: Prepare dataframes to load into the Database 

In [20]:
# Join best ever written and best seller data frames 

best_books = best_selling_df.merge(greatest_ever_written_df, on='title', how='outer')

# Eliminate NULLs from best_seller and best_ever_written columns (they should be boolean)
best_books_df = best_books.fillna({'best_seller': False,'best_ever_written': False,'authors': "",
                                  'original_language': 'Undefined','first_published': 9999, 'genre': 'Undefined'})
best_books_df

,title,authors,original_language,first_published,approximate_sales,genre,best_seller,best_ever_written
0,A Tale of Two Cities,Charles Dickens,English,1859,200,fiction,True,False
1,The Little Prince,Antoine de Saint-Exupéry,French,1943,150,fantasy,True,False
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120,fantasy,True,False
3,The Hobbit,J. R. R. Tolkien,English,1937,100,fantasy,True,False
4,And Then There Were None,Agatha Christie,English,1939,100,mystery,True,False
...,...,...,...,...,...,...,...,...
169,Don Quixote,,Undefined,9999,NaN,Undefined,False,True
170,Beloved,,Undefined,9999,NaN,Undefined,False,True
171,Mrs. Dalloway,,Undefined,9999,NaN,Undefined,False,True
172,Jane Eyre,,Undefined,9999,NaN,Undefined,False,True


In [21]:
#Join with ratings Data Frame to fill the missing authors 
best_ratings = best_books_df.merge(ratigns_df_clean, on='title', how='left')

for x in range(len(best_ratings)):
    if best_ratings.iloc[x,1] == "":
        best_ratings.iloc[x,1] = best_ratings.iloc[x,8]

best_books_final = best_ratings[['title','authors_x', 'original_language','first_published','approximate_sales','genre','best_ever_written','best_seller']].copy()
best_books_final.columns=['title','author', 'original_language','first_published','approximate_sales','genre','best_ever_written','best_seller']


best_books_final


,title,author,original_language,first_published,approximate_sales,genre,best_ever_written,best_seller
0,A Tale of Two Cities,Charles Dickens,English,1859,200,fiction,False,True
1,The Little Prince,Antoine de Saint-Exupéry,French,1943,150,fantasy,False,True
2,Harry Potter and the Philosopher's Stone,J. K. Rowling,English,1997,120,fantasy,False,True
3,The Hobbit,J. R. R. Tolkien,English,1937,100,fantasy,False,True
4,And Then There Were None,Agatha Christie,English,1939,100,mystery,False,True
...,...,...,...,...,...,...,...,...
169,Don Quixote,Miguel de Cervantes Saavedra,Undefined,9999,NaN,Undefined,True,False
170,Beloved,Toni Morrison,Undefined,9999,NaN,Undefined,True,False
171,Mrs. Dalloway,Virginia Woolf,Undefined,9999,NaN,Undefined,True,False
172,Jane Eyre,Charlotte Brontë,Undefined,9999,NaN,Undefined,True,False


In [22]:
# Original Language data frame 
language_list = best_books_final['original_language'].unique()
language_df = pd.DataFrame(language_list,columns=['original_language'])
language_df['id_language']=""

counter = 1000
for x in range(len(language_df)):
    counter += 1
    language_df.iloc[x,1]= f'l-{counter}'

language_df

,original_language,id_language
0,English,l-1001
1,French,l-1002
2,Chinese,l-1003
3,Italian,l-1004
4,Portuguese,l-1005
5,Spanish,l-1006
6,German,l-1007
7,Norwegian,l-1008
8,Russian,l-1009
9,Dutch,l-1010


In [23]:
# Authors data frame 
authors_list = best_books_final['author'].unique()
authors_df = pd.DataFrame(authors_list,columns=['author'])
authors_df['id_author']=""

counter = 1000
for x in range(len(authors_df)):
    counter += 1
    authors_df.iloc[x,1]= f'a-{counter}'

    
authors_df

,author,id_author
0,Charles Dickens,a-1001
1,Antoine de Saint-Exupéry,a-1002
2,J. K. Rowling,a-1003
3,J. R. R. Tolkien,a-1004
4,Agatha Christie,a-1005
...,...,...
152,Miguel de Cervantes Saavedra,a-1153
153,Toni Morrison,a-1154
154,Virginia Woolf,a-1155
155,Charlotte Brontë,a-1156


In [24]:
# Eliminate blank spaces 
authors_df['author']=authors_df['author'].str.strip()
authors_df['author'].str.len().max()

39

In [25]:
# Genre data frame 
genre_list = best_books_final['genre'].unique()
genre_df = pd.DataFrame(genre_list,columns=['genre'])
genre_df['id_genre']=""

counter = 1000
for x in range(len(genre_df)):
    counter += 1
    genre_df.iloc[x,1]= f'a-{counter}'

genre_df

,genre,id_genre
0,fiction,a-1001
1,fantasy,a-1002
2,mystery,a-1003
3,family saga,a-1004
4,adventure,a-1005
5,coming-of-age,a-1006
6,romance,a-1007
7,historical,a-1008
8,self-improvement,a-1009
9,magic realism,a-1010


In [26]:
# Eliminate blank spaces 
genre_df['genre']=genre_df['genre'].str.strip()
genre_df['genre'].str.len().max()

21

In [27]:
# Best Books Data Frame

best_books_final['id_title']=""

counter = 1000
for x in range(len(best_books_final)):
    counter += 1
    best_books_final.iloc[x,8]= f't-{counter}'
        
    for y in range(len(authors_df)):
        if authors_df.iloc[y,0] == best_books_final.iloc[x,1]:
            best_books_final.iloc[x,1] = authors_df.iloc[y,1]
            break
    
    for z in range(len(genre_df)):
        if genre_df.iloc[z,0] == best_books_final.iloc[x,5]:
            best_books_final.iloc[x,5] = genre_df.iloc[z,1]
            break
    
    for w in range(len(language_df)):
        if language_df.iloc[w,0] == best_books_final.iloc[x,2]:
            best_books_final.iloc[x,2] = language_df.iloc[w,1]
            break
    
best_books=best_books_final.rename(columns={'author':'id_author','original_language':'id_language','genre':'id_genre'})
best_books


,title,id_author,id_language,first_published,approximate_sales,id_genre,best_ever_written,best_seller,id_title
0,A Tale of Two Cities,a-1001,l-1001,1859,200,a-1001,False,True,t-1001
1,The Little Prince,a-1002,l-1002,1943,150,a-1002,False,True,t-1002
2,Harry Potter and the Philosopher's Stone,a-1003,l-1001,1997,120,a-1002,False,True,t-1003
3,The Hobbit,a-1004,l-1001,1937,100,a-1002,False,True,t-1004
4,And Then There Were None,a-1005,l-1001,1939,100,a-1003,False,True,t-1005
...,...,...,...,...,...,...,...,...,...
169,Don Quixote,a-1153,l-1017,9999,NaN,a-1015,True,False,t-1170
170,Beloved,a-1154,l-1017,9999,NaN,a-1015,True,False,t-1171
171,Mrs. Dalloway,a-1155,l-1017,9999,NaN,a-1015,True,False,t-1172
172,Jane Eyre,a-1156,l-1017,9999,NaN,a-1015,True,False,t-1173


In [28]:
# Eliminate blank spaces 
best_books['title']=best_books['title'].str.strip()
best_books['title'].str.len().max()

44

In [29]:
# Ratings Data Frame 
ratings_data = best_books.merge(ratigns_df_clean, on='title', how='inner')
ratings_data_final = ratings_data[['title','average_rating','ratings_count', 'text_reviews_count']].copy()
ratings_data_final.columns=['title','average_rating','ratings_count', 'text_reviews_count']

counter = 1000
ratings_data_final['id_rating']=""
for x in range(len(ratings_data_final)):
    counter += 1
    ratings_data_final.iloc[x,4]= f'r-{counter}'
    
    for y in range(len(best_books)):
        if best_books.iloc[y,0] == ratings_data_final.iloc[x,0]:
            ratings_data_final.iloc[x,0] = best_books.iloc[y,8]
            break

ratings_data_final=ratings_data_final.rename(columns={'title':'id_title'})
len(ratings_data_final)


65

In [30]:
ratings_data_final

,id_title,average_rating,ratings_count,text_reviews_count,id_rating
0,t-1001,3.84,1343,183,r-1001
1,t-1004,4.27,3213,329,r-1002
2,t-1005,4.26,625767,18785,r-1003
3,t-1010,3.84,13934,1459,r-1004
4,t-1017,3.86,1631221,55843,r-1005
...,...,...,...,...,...
60,t-1170,3.87,160911,4087,r-1061
61,t-1171,3.83,282045,9004,r-1062
62,t-1172,3.79,176190,5595,r-1063
63,t-1173,4.12,1409369,27884,r-1064


In [31]:
# Eliminate blank spaces 
ratings_data_final['id_title']=ratings_data_final['id_title'].str.strip()
ratings_data_final['id_title'].str.len().max()

6

## Create database connection

In [32]:
rds_connection_string = "postgres:Accm84$1@localhost:5432/best_books_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [33]:
engine.table_names()

['authors', 'best_books', 'original_languages', 'genres', 'ratings_data']

## Load Final Data Frames to Database

In [34]:
language_df.to_sql(name='original_languages', con=engine, if_exists='append', index=False)

In [35]:
authors_df.to_sql(name='authors', con=engine, if_exists='append', index=False)

In [36]:
genre_df.to_sql(name='genres', con=engine, if_exists='append', index=False)

In [37]:
best_books.to_sql(name='best_books', con=engine, if_exists='append', index=False)

In [38]:
ratings_data_final.to_sql(name='ratings_data', con=engine, if_exists='append', index=False)